In [31]:
import os
import torch
import json
import cv2
import numpy as np
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from albumentations import Compose, Resize, Normalize, ToFloat
from albumentations.pytorch import ToTensorV2

class COCOSegmentationDataset(Dataset):
    def __init__(self, root, annFile, transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        img_info = self.coco.loadImgs(img_id)[0]
        path = img_info['file_name']

        # Load image
        image = cv2.imread(os.path.join(self.root, path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load mask
        mask = np.zeros((img_info['height'], img_info['width']))
        for ann in anns:
            mask = np.maximum(self.coco.annToMask(ann), mask)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        mask = np.expand_dims(mask, axis=0)  # Add channel dimension
        return image, mask

transform = Compose([
    Resize(256, 256),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
    ToFloat(max_value = 100),
])

train_dataset = COCOSegmentationDataset(
    root= "data-lab3-coco/test/data",
    annFile="data-lab3-coco/test/labels.json",
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=7, shuffle=True)
# train_dataset[0][1].shape
iter = train_loader.__iter__()
batch1 = next(iter)
batch1 = next(iter)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


AttributeError: 'Tensor' object has no attribute 'astype'

In [26]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

class UNet(pl.LightningModule):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return torch.sigmoid(x)

    def training_step(self, batch, batch_idx):
        images, masks = batch
        preds = self(images)
        loss = nn.BCELoss()(preds, masks)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [28]:
from pytorch_lightning import Trainer

# Initialize model
model = UNet()

# Initialize a trainer
trainer = Trainer(max_epochs=10)

# Train the model
trainer.fit(model, train_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/domantas.keturakis/Projects/gmm/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: /Users/domantas.keturakis/Projects/gmm/lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 38.7 K
1 | decoder | Sequential | 16.5 K
---------------------------------------
55.2 K    Trainable params
0    

Training: |          | 0/? [00:00<?, ?it/s]

TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.